In [206]:
import json
import time
from selenium import webdriver
import pandas as pd
import matplotlib.pyplot as plt
from markdownify import markdownify
from time import sleep
from selenium.webdriver.common.action_chains import ActionChains

In [228]:
def scrape_job_links(driver,url,link_css_selector,stop_function,page_start=1,page_inc=1,):
    page = page_start
    
    job_links = []
    
    while True:

        driver.get(url.format(page_number=page))
        
        print(f'scraping {driver.current_url}')
        
        job_link_elements = driver.find_elements_by_css_selector(link_css_selector)
        
        if stop_function(driver) == True:
            break
        else:
            print(f'found {len(job_link_elements)} urls')
            job_links+=[job_link_element.get_attribute('href') for job_link_element in job_link_elements]
        
        page+=page_inc
        
    return list(set(job_links))

def scrape_cwjob_details(driver,url):
    driver.get(url)
    
    job = {}
    
    df_job = json.loads(driver.find_elements_by_css_selector('script#jobPostingSchema')[0].get_attribute('innerHTML'))
    
    job['url'] = df_job['url']
    job['title'] = df_job['title']
    job['job_type'] = df_job['employmentType']
    job['job_description_html'] = df_job['description']
    
    df_hiring_org = pd.json_normalize(df_job['hiringOrganization'])
    
    job['hiring_organization'] = df_hiring_org['name'][0]
    
    

    return job

def scroll_to_element(passed_in_driver, object):
    x = object.location['x']
    y = object.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % (
        x,
        y
    )
    scroll_nav_out_of_way = 'window.scrollBy(0, -120);'
    passed_in_driver.execute_script(scroll_by_coord)
    passed_in_driver.execute_script(scroll_nav_out_of_way)

def scrape_indeed_job_details(driver):
    """
    scrape job details from indeed.co.uk

    args:
        driver: selenium web driver

    returns:
        array of dictionary objects containing job details
    """
        
    page = 1

    jobs = []

    while True:
        driver.get(f'https://www.indeed.co.uk/jobs?q=data+scientist')
        
        driver.implicitly_wait(2)

        for link in driver.find_elements_by_css_selector('.jobtitle.turnstileLink'):
            scroll_to_element(driver,link)
            actions = ActionChains(driver)
            actions.move_to_element(link).click().perform()            

            job = {}

            job['url'] = url
            job['title'] = driver.find_element_by_css_selector('div#vjs-jobtitle').text
            job['hiring_organization'] = driver.find_element_by_css_selector('div#vjs-header-jobinfo div#vjs-jobinfo div span#vjs-cn').text
            job['location'] = driver.find_element_by_css_selector('div#vjs-jobinfo div span#vjs-loc').text
            try:
                job['remuneration'] = driver.find_elements_by_css_selector('#vjs-jobinfo > div:nth-child(3) > span:nth-child(1)')[0].text
            except:
                job['remuneration'] = None
            
            try:
                job['job_type'] = driver.find_elements_by_css_selector('#vjs-jobinfo > div:nth-child(3) > span:nth-child(2)')[0].text
            except:
                job['remuneration'] = None
                
            job['job_description_html'] = driver.find_elements_by_css_selector('#vjs-desc')[0].get_attribute('innerHTML')
            job['job_description'] = driver.find_elements_by_css_selector('#vjs-desc')[0].text

            jobs.append(job)

        page+=1
        
        break

    return pd.DataFrame(jobs)



In [229]:
driver = webdriver.Firefox()


jobs = scrape_indeed_job_details(driver)

In [231]:
jobs

,url,title,hiring_organization,location,remuneration,job_type,job_description_html,job_description
0,https://www.indeed.co.uk/jobs?q=data+scientist...,Junior Data Scientist,Fable Data Limited,- London,"£25,000 - £35,000 a year","Full-time, Permanent",<p><b>About Fable Data </b></p><p>Fable Data i...,About Fable Data\nFable Data is a leading alte...
1,https://www.indeed.co.uk/jobs?q=data+scientist...,Data Scientist,Beauty Matching Engine,- Home Based,"£31,372 - £88,825 a year","Full-time, Contract",<p>Role:</p><ul><li>This is a 3-6 month full-t...,Role:\nThis is a 3-6 month full-time data scie...
2,https://www.indeed.co.uk/jobs?q=data+scientist...,Senior Data Scientist,QVC,- West London,None,NaN,<p></p><div>Keeping millions of global custome...,Keeping millions of global customers engaged t...
3,https://www.indeed.co.uk/jobs?q=data+scientist...,Customer Data Scientist,New Look,- London W1W 8HJ,None,NaN,"<p></p><div><div><div><h2 class=""jobSectionHea...",CUSTOMER DATA SCIENTIST\nLevel\nL3 - Manager\n...
4,https://www.indeed.co.uk/jobs?q=data+scientist...,Data Scientist,First Response Finance Ltd,- Nottingham,None,NaN,<p>If data is your thing and you’re looking fo...,If data is your thing and you’re looking for s...
5,https://www.indeed.co.uk/jobs?q=data+scientist...,Data Scientist,Netacea,- Manchester,None,NaN,<p>Job Description Data Scientist</p><p>Netace...,Job Description Data Scientist\nNetacea provid...
6,https://www.indeed.co.uk/jobs?q=data+scientist...,Data Scientist Intern,Buzz Radar,- London,None,NaN,"<div></div><div><div><div><div><div>London, UK...","London, UK, Full Time.\nWe’re excited to be ab..."
7,https://www.indeed.co.uk/jobs?q=data+scientist...,Data Scientist - Graduate Entry,SeamlessML,- Cambridge,None,NaN,<div><p><b>Data Scientist - Graduate Entry</b>...,Data Scientist - Graduate Entry\nThe Data Scie...
8,https://www.indeed.co.uk/jobs?q=data+scientist...,Data Scientist,VaynerMedia LLC,- London,None,NaN,<div><p><b>VaynerMedia is a full-service globa...,VaynerMedia is a full-service global agency bu...
9,https://www.indeed.co.uk/jobs?q=data+scientist...,Data Scientist,QuEST Global Engineering,- London,None,NaN,<div></div><div><div>Required skillsets:<br>\n...,Required skillsets:\n\n1.Data Scientist - Time...


In [227]:
scroll_shim(driver,driver.find_elements_by_css_selector('.jobsearch-SerpJobCard.unifiedRow.row.result.clickcard')[2])
actions = ActionChains(driver)
actions.move_to_element(driver.find_elements_by_css_selector('.jobsearch-SerpJobCard.unifiedRow.row.result.clickcard')[2]).perform()